In [2]:
#hide
# default_exp core
from nbdev.showdoc import *

# kicad-helpers

> Scripts, templates, and examples for managing KiCad projects.

In [3]:
#export
import glob
import os
import subprocess
import tempfile

import git
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

In [4]:
#export
def get_git_root(path="."):
    # Find the current projects' root directory
    git_repo = git.Repo(path, search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [5]:
#export
# Set the PROJECT ROOT environment variable
#PROJECT_ROOT = get_git_root()
#os.environ["KH_PROJECT_ROOT"] = PROJECT_ROOT
#assert(os.getenv("KH_PROJECT_ROOT") == PROJECT_ROOT)

The `$KH_PROJECT_ROOT` environment variable gets automatically set when the kicad_helpers package is imported making it accessible to shell scripts. `PROJECT_ROOT` is also available as a python variable in the `kicad_helpers` module.

# Project metadata/configuration

Metadata and configuration for the project are stored in the `.kicad_helpers_config` directory (including [kibot](https://github.com/INTI-CMNB/KiBot) config files for producting manufacturing outputs via continuous integration). Additional metadata (e.g., project summary, website, and manufacturing details) is stored in the project's `kitspace.yaml` file which makes it easy for other people to manufacture boards using https://kitspace.org/.

In [22]:
#export
def _set_root(root):
    if root == ".":
        # Use defaults
        root = get_git_root(".")
        
        # Override with environment variable if set
        root = os.getenv("KH_PROJECT_ROOT", root)
    return root

def get_project_name(root="."):
    # This is based on the name of the KiCad `*.pro` file.
    root = _set_root(root)
    return os.path.splitext(os.path.split(glob.glob(os.path.join(root, "*.pro"))[0])[1])[0]

def get_project_metadata(root="."):
    root = _set_root(root)
    # Default metadata if there's no existing `kicad.yaml` file.
    metadata = {"summary": "A description for your project",
                "site": "https://example.com # A site you would like to link to (include http:// or https://)",
                "color": "black"
    }
    
    try:
        # If there's an existing `kicad.yaml` file, those settings override the defaults.
        with open(os.path.join(root, "kitspace.yaml")) as f:
            metadata.update({k: v for k, v in load(f, Loader=Loader).items() if k in ["summary", "site", "color"]})
    except FileNotFoundError:
        pass

    # Add the project name
    metadata["project_name"]=get_project_name(root)
    return metadata

def get_schematic_path(root="."):
    root = _set_root(root)
    return os.path.join(root, get_project_name(root) + ".sch")

def get_bom_path(root="."):
    return os.path.join(root, "manufacturing", "default", get_project_name(root) + "-BOM.csv")

In [7]:
#export
def setup_test_repo(root):
    subprocess.check_call(f"git clone https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad { root }", shell=True)

In [8]:
temp_dir = tempfile.mkdtemp()
setup_test_repo(temp_dir)
print(f"get_project_name() = { get_project_name(temp_dir) }")
print(f"get_project_metadata() = { get_project_metadata(temp_dir) }")
print(f"get_schematic_path() = { get_schematic_path(temp_dir) }")
print(f"get_bom_path() = { get_bom_path(temp_dir) }")
assert get_project_name(temp_dir) == "40-channel-hv-switching-board"
assert get_project_metadata(temp_dir) == {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}

Cloning into '/tmp/tmpo6n6gt92'...


get_project_name() = 40-channel-hv-switching-board
get_project_metadata() = {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}
get_schematic_path() = /tmp/tmpo6n6gt92/40-channel-hv-switching-board.sch


Updating files: 100% (58/58), done.


NameError: name 'manufacturing' is not defined

In [24]:
print(f"get_project_name() = { get_project_name(temp_dir) }")
print(f"get_project_metadata() = { get_project_metadata(temp_dir) }")
print(f"get_schematic_path() = { get_schematic_path(temp_dir) }")
print(f"get_bom_path() = { get_bom_path(temp_dir) }")
assert get_project_name(temp_dir) == "40-channel-hv-switching-board"
assert get_project_metadata(temp_dir) == {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}
assert os.path.exists(get_schematic_path(temp_dir))
assert os.path.exists(get_bom_path(temp_dir))

get_project_name() = 40-channel-hv-switching-board
get_project_metadata() = {'summary': 'DropBot v3 40-channel high-voltage switching board', 'site': 'https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad', 'color': 'black', 'project_name': '40-channel-hv-switching-board'}
get_schematic_path() = /tmp/tmpo6n6gt92/40-channel-hv-switching-board.sch
get_bom_path() = /tmp/tmpo6n6gt92/manufacturing/default/40-channel-hv-switching-board-BOM.csv


In [ ]:
#export
def get_gitignore_list(root="."):
    root =_set_root(root)
    with open(f"{ root }/.gitignore") as f:
        gitignore = f.readlines()
    return "|".join([line.strip() for line in gitignore])

def in_gitignore(filename):
    try:
        if len(subprocess.check_output(f"echo '{ filename }' | git check-ignore --stdin --no-index", shell=True)):
            return True
    except subprocess.CalledProcessError:
        pass
    return False